<a href="https://colab.research.google.com/github/enya-yx/LangChain-Courses/blob/main/tool_and_routing_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install "langchain-google-genai" "langchain" "langchain-core" "langgraph-prebuilt" "google-generativeai" "langchain_community" "docarray" "langchain_experimental"

INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langgraph-prebuilt to determine which version is compatible with other requirements. This could t

In [2]:
import google.generativeai as genai
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('google_api_key')
# Configure the generative AI library with your API key
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])


In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Define llm
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    verbose=True
)


In [4]:
# Define tools by decorator and using pandatic class as args
from typing import List
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain.agents import tool

class SearchInput(BaseModel):
  query: str = Field(description="thing to search for")

@tool(args_schema=SearchInput)
def search(query: str) -> str:
  """Search for weather online"""
  return "42f"
search.args

{'query': {'description': 'thing to search for',
  'title': 'Query',
  'type': 'string'}}

In [5]:
# Define the first tool to query current temperature for a position
import requests
class Position(BaseModel):
  latitude: float = Field(description="latitude of the location")
  longitude: float = Field(description="longitude of the location")

@tool(args_schema=Position)
def get_current_temperature(latitude: float, longitude: float) -> str:
  """Get the current temperature"""

  BASE_URL = "https://api.open-meteo.com/v1/forecast"
  params = {
    "latitude": latitude,
    "longitude": longitude,
    "hourly": "temperature_2m",
    "forecast_days": 1,
  }
  response = requests.get(BASE_URL, params=params)
  if response.status_code == 200:
    data = response.json()
    hourly_data = data["hourly"]
    temperature_2m = hourly_data["temperature_2m"]
    unit = data["hourly_units"]["temperature_2m"]

    return f"The current temperature is: {temperature_2m[0]}{unit}"
  else:
    raise Exception(f"Request failed with status code {response.status_code}")


get_current_temperature.invoke({"latitude": 13, "longitude":14})

'The current temperature is: 21.3°C'

In [6]:
from langchain.tools.render import format_tool_to_openai_function
get_current_temperature_function = format_tool_to_openai_function(get_current_temperature)
get_current_temperature_function


/tmp/ipython-input-1714603674.py:2: LangChainDeprecationWarning: The function `_format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use :meth:`~langchain_core.utils.function_calling.convert_to_openai_function()` instead.
  get_current_temperature_function = format_tool_to_openai_function(get_current_temperature)


{'name': 'get_current_temperature',
 'description': 'Get the current temperature',
 'parameters': {'properties': {'latitude': {'description': 'latitude of the location',
    'type': 'number'},
   'longitude': {'description': 'longitude of the location',
    'type': 'number'}},
  'required': ['latitude', 'longitude'],
  'type': 'object'}}

In [7]:
# Define prompt and bind function to llm
template_string = "What is the current temperature at position latitude: {latitude}, longitude: {longitude}?"
prompt = PromptTemplate(
    input_variables = ["latitude", "longitude"],
    template=template_string
)
#message_test = prompt.format(latitude=13, longitude=14)
#message_test

llm_with_function = llm.bind(functions = [get_current_temperature_function])

chain = prompt | llm_with_function
res = chain.invoke({"latitude": 13, "longitude":14})
print(res)

content='' additional_kwargs={'function_call': {'name': 'get_current_temperature', 'arguments': '{"latitude": 13.0, "longitude": 14.0}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run--c500f42d-7e12-4aaa-8b22-6fee19445c55-0' tool_calls=[{'name': 'get_current_temperature', 'args': {'latitude': 13.0, 'longitude': 14.0}, 'id': '1488a011-7d96-45d9-baa1-4d2aeed1322e', 'type': 'tool_call'}] usage_metadata={'input_tokens': 75, 'output_tokens': 24, 'total_tokens': 177, 'input_token_details': {'cache_read': 0}}


In [8]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=8783874a406d08f54cd7373e652765003eae80e98fa786c3ed39ff333753a666
  Stored in directory: /root/.cache/pip/wheels/63/47/7c/a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia


In [9]:
# Define the second tool to use wikipedia
import wikipedia
@tool
def search_wikipedia(query: str) -> str:
  """Search wikipedia for the given query and get page summaries"""
  page_titles = wikipedia.search(query)
  summaries = []
  for page_title in page_titles[:3]:
    try:
      wiki_pedia_page = wikipedia.page(page_title)
      summaries.append(f"Page: {page_title}\nSummary: {wiki_pedia_page.summary}")
    except wikipedia.exceptions.DisambiguationError as e:
      print(f"Skipping disambiguation page: {page_title}")

  if not summaries:
    return "No good Wikipedia Search Result was found"

  return "\n\n".join(summaries)

search_wikipedia_function = format_tool_to_openai_function(search_wikipedia)

In [10]:
# bind model with multiple functions
llm_multi_function = llm.bind(functions = [get_current_temperature_function, search_wikipedia_function])
res1 = llm_multi_function.invoke("What is the current weather at the position: latitude 13, longitide 14 right now?")
res2 = llm_multi_function.invoke("Give a short introduction about the game Werewolves Kill")
print(res1)
print(res2)


content='' additional_kwargs={'function_call': {'name': 'get_current_temperature', 'arguments': '{"latitude": 13.0, "longitude": 14.0}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run--6c79c419-c426-451e-9d97-e3f2e0635b3c-0' tool_calls=[{'name': 'get_current_temperature', 'args': {'latitude': 13.0, 'longitude': 14.0}, 'id': '0e65314e-bd4d-41ad-b393-564232ca5b25', 'type': 'tool_call'}] usage_metadata={'input_tokens': 121, 'output_tokens': 24, 'total_tokens': 230, 'input_token_details': {'cache_read': 0}}
content='' additional_kwargs={'function_call': {'name': 'search_wikipedia', 'arguments': '{"query": "Werewolves Kill game"}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run--b3af06a3-1382-4d68-9860-33b6fdf2996e-0' tool_calls=[{'name': 'search_wikipedia', 'args': {'query': 'Werewolves Kill game'}, 'id': '1da129

In [117]:
# Define the executor as a rounter to trigger tools or return the content
from langchain_core.runnables import RunnableLambda
from langchain_core.messages import BaseMessage

def tool_executor(message: BaseMessage):
    if message.tool_calls:
        tool_call = message.tool_calls[0]
        tool_name = tool_call['name']
        tool_args = tool_call['args']
        tools = {
          "get_current_temperature": get_current_temperature,
          "search_wikipedia": search_wikipedia
        }

        # Assuming get_current_temperature is accessible in the scope
        if tool_name in tools:
            return tools[tool_name].invoke(tool_args)
        else:
            # Handle other tools if necessary, or raise an error
            return f"Unknown tool: {tool_name}"
    else:
        return message.content # Return the LLM's content if no tool call

tool_execution_runnable = RunnableLambda(tool_executor)

print("Defined tool_executor function and tool_execution_runnable.")

Defined tool_executor function and tool_execution_runnable.


In [119]:
# Add tool executor to the chain to trigger the function automatically
# from langchain.agents.output_parsers import JsonOutputFunctionsParser
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}")]
)

full_chain = prompt | llm_multi_function | tool_execution_runnable

'The current temperature is: 24.4°C'

In [ ]:
full_chain.invoke({"input": "What is the current weather at the position: latitude 13, longitide 14 right now?"})

In [120]:
full_chain.invoke({"input": "Hi, I'm Yx"})

'Hello Yx! Nice to meet you. How can I help you today?'

In [121]:
full_chain.invoke({"input": "What is the game of Werewolves Kill?"})

'Page: Mafia (party game)\nSummary: Mafia, also known as Werewolf, is a social deduction game created in 1986 by Dimitry Davidoff, then a psychology student at Moscow State University. The game models a conflict between two groups: an informed minority (the mafiosi or the werewolves) and an uninformed majority (the villagers). At the start of the game, each player is secretly assigned a role affiliated with one of these teams. The game has two alternating phases: first, a night-phase, during which those with night-killing-powers may covertly kill other players, and second, a day-phase, in which all surviving players debate and vote to eliminate a suspect. The game continues until a faction achieves its win condition; for the village, this usually means eliminating the evil minority, while for the minority, this usually means reaching numerical parity with the village and eliminating any rival evil groups.\n\n\n\nPage: Werewolves Within (film)\nSummary: Werewolves Within is a 2021 Ameri

In [12]:
# Agent; Apply 'MessagePlaceholder' when define prompt to pass information from each chain invoke
from langchain.prompts import MessagesPlaceholder
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
prompt_with_holder = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scrachpad"),
])
chain = prompt_with_holder | llm_multi_function | OpenAIFunctionsAgentOutputParser()

In [26]:
result1 = chain.invoke({"input": "What is the game of Werewolves Kill?", "agent_scrachpad": []})


In [22]:
result1.message_log

[AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_wikipedia', 'arguments': '{"query": "Werewolves Kill game"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--102a31da-029a-4d9d-9cd7-5351e7e53a3c-0', tool_calls=[{'name': 'search_wikipedia', 'args': {'query': 'Werewolves Kill game'}, 'id': '33be783c-d0f6-4292-859f-1f9b144f788d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 112, 'output_tokens': 18, 'total_tokens': 165, 'input_token_details': {'cache_read': 0}})]

In [19]:
func = result1.tool
obersavation = search_wikipedia.invoke(result1.tool_input)
print(obersavation)

Page: Mafia (party game)
Summary: Mafia, also known as Werewolf, is a social deduction game created in 1986 by Dimitry Davidoff, then a psychology student at Moscow State University. The game models a conflict between two groups: an informed minority (the mafiosi or the werewolves) and an uninformed majority (the villagers). At the start of the game, each player is secretly assigned a role affiliated with one of these teams. The game has two alternating phases: first, a night-phase, during which those with night-killing-powers may covertly kill other players, and second, a day-phase, in which all surviving players debate and vote to eliminate a suspect. The game continues until a faction achieves its win condition; for the village, this usually means eliminating the evil minority, while for the minority, this usually means reaching numerical parity with the village and eliminating any rival evil groups.



Page: Werewolves Within (film)
Summary: Werewolves Within is a 2021 American com

In [25]:
from langchain_core.messages import ToolMessage

# result1 is an AgentActionMessageLog object. Its message_log contains the AIMessage.
# obersavation is the string result from the tool execution.

# Construct the agent_scrachpad correctly:
# 1. The AIMessage from the LLM that suggested the tool call
# 2. A ToolMessage containing the observation (the result of the tool execution)
agent_scratchpad_messages = [
    result1.message_log[0], # The AIMessage from the LLM
    ToolMessage(content=obersavation, tool_call_id=result1.message_log[0].tool_calls[0]['id']) # The result of the tool execution
]

result2 = chain.invoke({
    "input": "What is the game of Werewolves Kill?",
    "agent_scrachpad": agent_scratchpad_messages
})

return_values={'output': '"Werewolves Kill" is likely a reference to the social deduction game "Mafia," also known as "Werewolf." In this game, players are secretly assigned roles as either an informed minority (werewolves/mafia) or an uninformed majority (villagers). The game alternates between a "night" phase where werewolves secretly "kill" players, and a "day" phase where all surviving players debate and vote to eliminate a suspect. The goal for the villagers is to eliminate the werewolves, while the werewolves aim to achieve numerical parity with the villagers.'} log='"Werewolves Kill" is likely a reference to the social deduction game "Mafia," also known as "Werewolf." In this game, players are secretly assigned roles as either an informed minority (werewolves/mafia) or an uninformed majority (villagers). The game alternates between a "night" phase where werewolves secretly "kill" players, and a "day" phase where all surviving players debate and vote to eliminate a suspect. The g

In [27]:
result1

AgentActionMessageLog(tool='search_wikipedia', tool_input={'query': 'Werewolves Kill'}, log="\nInvoking: `search_wikipedia` with `{'query': 'Werewolves Kill'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_wikipedia', 'arguments': '{"query": "Werewolves Kill"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--481a333c-5366-4e85-a0fb-c7de66636ab2-0', tool_calls=[{'name': 'search_wikipedia', 'args': {'query': 'Werewolves Kill'}, 'id': '7b126815-a88e-4e3e-8b18-61ef464fc2bf', 'type': 'tool_call'}], usage_metadata={'input_tokens': 112, 'output_tokens': 17, 'total_tokens': 164, 'input_token_details': {'cache_read': 0}})])

In [28]:
result2

AgentFinish(return_values={'output': '"Werewolves Kill" is likely a reference to the social deduction game "Mafia," also known as "Werewolf." In this game, players are secretly assigned roles as either an informed minority (werewolves/mafia) or an uninformed majority (villagers). The game alternates between a "night" phase where werewolves secretly "kill" players, and a "day" phase where all surviving players debate and vote to eliminate a suspect. The goal for the villagers is to eliminate the werewolves, while the werewolves aim to achieve numerical parity with the villagers.'}, log='"Werewolves Kill" is likely a reference to the social deduction game "Mafia," also known as "Werewolf." In this game, players are secretly assigned roles as either an informed minority (werewolves/mafia) or an uninformed majority (villagers). The game alternates between a "night" phase where werewolves secretly "kill" players, and a "day" phase where all surviving players debate and vote to eliminate a s

In [38]:
# Define function to run steps based on return type
from langchain.schema import AgentFinish
tools = {
        "get_current_temperature": get_current_temperature,
        "search_wikipedia": search_wikipedia
  }
def run_agent(user_input):
  intermediate_steps = []
  while True:
    result = chain.invoke({"input": user_input, "agent_scrachpad": intermediate_steps})
    if isinstance(result, AgentFinish):
      return result
    else:
      tool = tools[result.tool]
      obersavation = tool.run(result.tool_input)
      intermediate_steps = [
        result.message_log[0], # The AIMessage from the LLM
        ToolMessage(content=obersavation, tool_call_id=result.message_log[0].tool_calls[0]['id']) # The result of the tool execution
      ]


In [30]:
run_agent("What is the current weather at the position: latitude 13, longitide 14 right now?")

AgentFinish(return_values={'output': 'The current temperature is 21.3°C.'}, log='The current temperature is 21.3°C.')

In [35]:
# Pending: Create a conversation agent with memory
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import MessagesPlaceholder, ChatPromptTemplate
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scrachpad"),
])
chain = prompt | llm | OpenAIFunctionsAgentOutputParser()
agent_chain = RunnablePassthrough.assign(agent_scrachpad = lambda x: format_tool_to_openai_function(x["intermediate_steps"]))